In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
import matplotlib.dates as mdates
from scipy import stats
warnings.filterwarnings("ignore")

def read_wl_csv(file_path):
    # Directly skip the last 6 footer lines and parse the first column as dates.
    # Also mark invalid values as NaN during read.
    wl_df = pd.read_csv(
        file_path,
        parse_dates=[0],
        na_values=[-999, -99, 99, 'NA', 'RM'],
        skipfooter=6,     # Skips the last 6 lines
        engine='python'   
    )

    wl_df.columns = ['date', 'pwl', 'bwl', 'harmwl']

    # Calculate surges
    wl_df['pwl surge'] = wl_df['pwl'] - wl_df['harmwl']
    wl_df['bwl surge'] = wl_df['bwl'] - wl_df['harmwl']

    return wl_df

def locate_gaps(WL_data):
    # Boolean mask where 'pwl' is NaN
    mask = WL_data['pwl'].isna()
    
    # Identify the start of each gap
    # A gap starts when current row is NaN but previous row is not NaN.
    gap_starts = mask & ~mask.shift(fill_value=False)
    
    # Identify the end of each gap
    # A gap ends when current row is NaN but next row is not NaN.
    gap_ends = mask & ~mask.shift(-1, fill_value=False)
    
    # Get the indices of the start and end of each gap
    start_indices = WL_data.index[gap_starts]
    end_indices = WL_data.index[gap_ends]
    
    # Calculate gap lengths (number of consecutive NaN entries)
    gap_lengths = end_indices.values - start_indices.values + 1
    
    # Get corresponding start dates of each gap
    start_dates = WL_data.loc[start_indices, 'date'].reset_index(drop=True)
    
    # Create the resulting DataFrame
    WL_data_gaps = pd.DataFrame({
        'date': start_dates,
        'gapLength': gap_lengths
    })
    # Calculate total gap time in minutes (assuming each row represents 6 minutes)
    WL_data_gaps['gapTime(min)'] = WL_data_gaps['gapLength'] * 6
    
    return WL_data_gaps

def eligible_gap_length(WL_gaps):
    # Select gaps of length exactly 1
    linear_gaps = WL_gaps[WL_gaps['gapLength'] == 1]
    
    # Select gaps of length >1 but <= 576
    gaps_less_5_days = WL_gaps[(WL_gaps['gapLength'] > 1) & (WL_gaps['gapLength'] <= 576)]

    return linear_gaps, gaps_less_5_days

def linear_fill(Wl_data, linear_gaps):
    # If there are no single-length gaps, just return the original data
    if linear_gaps.empty:
        print('No single gaps to fill')
        return Wl_data

    # Get the indices of the rows in Wl_data that correspond to the single-gap dates
    matching_idx = Wl_data[Wl_data['date'].isin(linear_gaps['date'])].index

    # Calculate the new values in a vectorized manner
    prev_surge = Wl_data.loc[matching_idx - 1, 'pwl surge'].values
    next_surge = Wl_data.loc[matching_idx + 1, 'pwl surge'].values
    harmwl_values = Wl_data.loc[matching_idx, 'harmwl'].values

    new_values = ((prev_surge + next_surge) / 2) + harmwl_values

    # Assign the calculated values back to the 'pwl' column
    Wl_data.loc[matching_idx, 'pwl'] = new_values

    return Wl_data


def check_bwl(Wl_data, gaps):
    if gaps.empty:
        print('No gaps available to fill')
        return gaps

    # Extract the indexes and lengths of the gaps
    matching_idx = Wl_data[Wl_data['date'].isin(gaps['date'])].index
    gap_lengths = gaps['gapLength'].values

    # Check each gap for completeness in the 'bwl' column
    valid_gaps = [
        Wl_data['bwl'].iloc[start: start + length].notna().all()
        for start, length in zip(matching_idx, gap_lengths)
    ]

    # Filter gaps based on whether 'bwl' had no missing values
    filtered_gaps = gaps[valid_gaps].reset_index(drop=True)

    return filtered_gaps


def poly_gap_fill(Wl_data, gaps):
    # If no gaps, no need to process further
    if gaps.empty:
        print('No gaps to Fill')
        return [], [], [], [], []

    # Extract relevant arrays and indices
    Wl_date = Wl_data['date'].values
    Wl_pwl = Wl_data['pwl'].values
    Wl_bwl = Wl_data['bwl'].values
    Wl_index = Wl_data.index

    # Extract gaps info
    gap_dates = gaps['date'].values
    gap_lengths = gaps['gapLength'].values

    # Identify indices in Wl_data that correspond to the gap start dates
    # Since we're using isin, which might not preserve order perfectly in some edge cases,
    # we assume that the order of gap_dates corresponds to the same order in Wl_data.
    # If order is critical, you might need a more robust approach.
    matching_mask = np.isin(Wl_date, gap_dates)
    index_locations = Wl_index[matching_mask].to_numpy()

    poly_df_list = []
    gap_date_list = []
    poly_filled_gaps = []

    # Create a list of gap-date sub-dataframes (just the dates)
    for start_idx, length in zip(index_locations, gap_lengths):
        gap_date_df = pd.DataFrame({
            'date': Wl_date[start_idx:start_idx + length]
        })
        gap_date_list.append(gap_date_df)

    # Process each gap
    for (start_idx, length) in zip(index_locations, gap_lengths):
        # Bound check for the ±2160 window
        left_bound = start_idx - 2160
        right_bound = start_idx + 2160 + length
        
        if left_bound < 0 or right_bound > len(Wl_data):
            print('Cannot fill gap out of bounds')
            continue

        pwl_30 = Wl_pwl[left_bound:right_bound]
        bwl_30 = Wl_bwl[left_bound:right_bound]
        dates_30 = Wl_date[left_bound:right_bound]

        # Create a temporary DataFrame
        linear_df = pd.DataFrame({
            'pwl 30': pwl_30,
            'bwl 30': bwl_30,
            'dates': pd.to_datetime(dates_30)
        })
        linear_df.dropna(inplace=True)

        if linear_df.empty:
            print('Cannot fill gap, no valid data in 30-day window')
            continue

        # Linear regression (bwl_30 vs pwl_30)
        slope, intercept, *_ = stats.linregress(linear_df['bwl 30'], linear_df['pwl 30'])

        poly_df = pd.DataFrame({
            'bwl': bwl_30,
            'pwl': pwl_30,
            'date': pd.to_datetime(dates_30)
        })

        poly_df['mwl linear'] = intercept + slope * poly_df['bwl']

        # Mask where linear fit differs from actual pwl by more than 0.1
        mask = np.abs(poly_df['mwl linear'] - poly_df['pwl']) > 0.1
        poly_df.loc[mask, ['pwl', 'mwl linear', 'bwl']] = np.nan

        # Check if there are enough valid points
        if poly_df['bwl'].isna().sum() + poly_df['pwl'].isna().sum() < len(Wl_data) * 0.1:
            poly_df_copy = poly_df.dropna().copy()
            if poly_df_copy.empty:
                print('Cannot fill gap, no valid data after filtering')
                continue

            # Fit polynomial of degree 4 (pwl as a function of bwl)
            # Note: We're fitting pwl vs bwl but naming poly1 accordingly.
            coeffs = np.polyfit(poly_df_copy['bwl'], poly_df_copy['pwl'], 4)
            poly1 = np.poly1d(coeffs)
            pred_values = poly1(poly_df['bwl'])

            poly_df['mwl'] = pred_values
            poly_df_list.append(poly_df)
            poly_filled_gaps.append((start_idx, length))
        else:
            print('Cannot fill gap, not enough valid points after filtering')

    return poly_df_list, index_locations, gap_lengths, gap_date_list, poly_filled_gaps

def fill_gaps(poly_list, gap_dates_list, wl_df, poly_gap_list):
    # Add mwl column if not already present
    if 'mwl' not in wl_df.columns:
        wl_df['mwl'] = np.nan

    # For each gap to be filled
    for i, (idx, length) in enumerate(poly_gap_list):
        poly_df = poly_list[i]

        # Compute start and end indices for poly_df slice
        start = 2160
        end = 2160 + length

        # Extract gap values as a numpy array
        gap_values = poly_df['mwl'].iloc[start:end].to_numpy()

        # Assign these values back to wl_df using iloc for clarity
        wl_df.iloc[idx:idx+length, wl_df.columns.get_loc('mwl')] = gap_values

    return wl_df


def adjustment(filled_df, poly_gaps):


    filled_df['mwl adjusted'] = np.nan
    idx, length = map(list, zip(*poly_gaps))

    for i in range(len(idx)):
        adjustment_values = []

        # Calculate averages before and after the gap
        average_before = np.nanmean(filled_df['pwl'][idx[i] - 6:idx[i]])
        average_after = np.nanmean(filled_df['pwl'][idx[i] + length[i]:idx[i] + length[i] + 6])

        n_length = length[i]

        for k in range(n_length):
            value = (average_after + (k+1 / n_length)) * (average_before - average_after)
            adjustment_values.append(value)

        filled_df.loc[idx[i]:idx[i] + length[i] - 1, 'mwl adjusted'] = (
            filled_df.loc[idx[i]:idx[i] + length[i] - 1, 'mwl'] + adjustment_values
        )

    filled_df['new wl adjustment'] = filled_df['pwl'].combine_first(filled_df['mwl adjusted'])
    filled_df['new wl'] = filled_df['pwl'].combine_first(filled_df['mwl'])
    return filled_df

        

def create_gaps(dataset):

    import random

    wl_data =  dataset.copy()

    random_index = [random.randint(0,len(wl_data))for _ in range(1000)]

    max_gap_size = 100
    random_index = random.sample(range(len(wl_data) - max_gap_size), 1000)


    #create one six min gap

    wl_data.loc[random_index[0], 'pwl'] = np.nan
    random_index = random_index[1:]

    # create 5 30 min gaps

    for i in range(4):

        wl_data.loc[random_index[i]:random_index[i] + 4, 'pwl'] = np.nan
    
    #random_index = random_index[100:]

    #create 10 1hr gaps

    for i in range(10):

        wl_data.loc[random_index[i]:random_index[i] + 9, 'pwl'] = np.nan
    
    #random_index = random_index[10:]

    #creates 50 5 hr gaps

    for i in range(50):

        wl_data.loc[random_index[i]:random_index[i] + 49, 'pwl'] = np.nan
    
    random_index = random_index[50:]

    #creates 100 10hr gaps

    for i in range(100):

        wl_data.loc[random_index[i]:random_index[i] + 99, 'pwl'] = np.nan
    
    #random_index = random_index[10:]

    return wl_data

def cbi_gapfill(filepath):

    print('Reading dataset')
    wl_dataset = read_wl_csv(filepath)

    wl_dataset_gaps = create_gaps(wl_dataset)

    print('Gaps Created')

    Wl_gaps = locate_gaps(wl_dataset_gaps)

    print('Total number of gaps: ', len(Wl_gaps))

    linear_gaps,multi_gaps = eligible_gap_length(Wl_gaps)

    print('Number of Linear Gaps filled:', len(linear_gaps))

    dataset_LF = linear_fill(wl_dataset_gaps,linear_gaps)

    print('Single gaps filled')

    valid_multi_gaps = check_bwl(dataset_LF,multi_gaps)

    print('Number of gaps with backup water level:', len(valid_multi_gaps))

    if len(valid_multi_gaps) > 0:

        poly_wl_list, index_location, gap_length, gap_list, poly_gap_list = poly_gap_fill(dataset_LF,valid_multi_gaps)


        if len(poly_wl_list) > 0 :

            filled_df = fill_gaps(poly_wl_list,gap_list,dataset_LF,poly_gap_list)

            filled_df = adjustment(filled_df, poly_gap_list)

            print('Gaps filled', + len(poly_wl_list))

            return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list
        
        else:

            return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list
    else:
        return wl_dataset,wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list


    '''gaps_true = input('Do you want to create artifical gaps y/n? ')

    if str(gaps_true) == str('y'):
        
        wl_dataset_gaps = create_gaps(wl_dataset)

        print('Gaps Created')

        Wl_gaps = locate_gaps(wl_dataset_gaps)

        print('Total number of gaps: ', len(Wl_gaps))

        linear_gaps,multi_gaps = eligible_gap_length(Wl_gaps)

        print('Number of Linear Gaps filled:', len(linear_gaps))

        dataset_LF = linear_fill(wl_dataset_gaps,linear_gaps)

        print('Single gaps filled')

        valid_multi_gaps = check_bwl(dataset_LF,multi_gaps)

        print('Number of gaps with backup water level:', len(valid_multi_gaps))

        if len(valid_multi_gaps) > 0:

            poly_wl_list, index_location, gap_length, gap_list, poly_gap_list = poly_gap_fill(dataset_LF,valid_multi_gaps)


            if len(poly_wl_list) > 0 :

                filled_df = fill_gaps(poly_wl_list,gap_list,dataset_LF,poly_gap_list)

                filled_df = adjustment(filled_df, poly_gap_list)

                print('Gaps filled', + len(poly_wl_list))

                return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list
            
            else:

                return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list
        else:
            return wl_dataset,wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list

    elif str(gaps_true) == str('n'):

        Wl_gaps = locate_gaps(wl_dataset)

        print('Total number of gaps: ', len(Wl_gaps))

        linear_gaps,multi_gaps = eligible_gap_length(Wl_gaps)

        print('Number of Linear Gaps filled:', len(linear_gaps))

        dataset_LF = linear_fill(wl_dataset,linear_gaps)

        print('Single gaps filled')

        valid_multi_gaps = check_bwl(dataset_LF,multi_gaps)

        print('Number of gaps with backup water level:', len(valid_multi_gaps))
        poly_wl_list, index_location, gap_length, gap_list, poly_gap_list = poly_gap_fill(dataset_LF,valid_multi_gaps)

        if len(valid_multi_gaps) > 0:

            poly_wl_list, index_location, gap_length, gap_list, poly_gap_list = poly_gap_fill(dataset_LF,valid_multi_gaps)


            if len(poly_wl_list) > 0 :

                filled_df = fill_gaps(poly_wl_list,gap_list,dataset_LF,poly_gap_list)

                filled_df = adjustment(filled_df, poly_gap_list)

                print('Gaps filled', + len(poly_wl_list))

                return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list
            
            else:

                return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list
        else:
            return wl_dataset,wl_dataset,Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list
    else:
        print('Not an acceptable answer')
        dataset_LF = []
        Wl_gaps = []
        dataset_LF = []
        poly_wl_list = []
        gap_list = []
        poly_gap_list = []
        
        return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, poly_gap_list'''


In [84]:
filepath = r'C:\Users\mrpro\Documents\Code\CBI\Gap_Filling\pd_1732232133.csv'

# Lists to store the results of each run
mean_error_list = []
mean_adjusted_error_list = []
mean_difference_list = []

num_runs = 10

for i in range(num_runs):
    print(f"Run {i+1}/{num_runs}")
    filled_df, wl_dataset, all_gaps, dataset_LF, poly_wl, filled_gap_list, adjusted_gaps = cbi_gapfill(filepath)
    
    # Calculate error metrics
    filled_df['pwl actual'] = wl_dataset['pwl']
    filled_df['error'] = np.abs((filled_df['mwl'] - filled_df['pwl actual']) / filled_df['pwl actual']) * 100
    filled_df['difference'] = np.abs(filled_df['pwl actual'] - filled_df['mwl'])

    filled_df = filled_df.round(3)
    # Compute and store the desired statistics
    mean_error_list.append(filled_df['error'].mean())
    mean_difference_list.append(filled_df['difference'].mean())

# After completing all runs, print or analyze the results
print("Mean Error per run:", mean_error_list)
print("Mean Difference per run:", mean_difference_list)

# You could also summarize across all runs if desired
print("Overall Mean Error:", round(np.mean(mean_error_list),3))
print("Overall Mean Difference:", round(np.mean(mean_difference_list),3))


Run 1/10
Reading dataset
Gaps Created
Total number of gaps:  162
Number of Linear Gaps filled: 8
Single gaps filled
Number of gaps with backup water level: 126
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Gaps filled 117
Run 2/10
Reading dataset
Gaps Created
Total number of gaps:  153
Number of Linear Gaps filled: 7
Single gaps filled
Number of gaps with backup water level: 120
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Gaps filled 117
Run 3/10
Reading dataset
Gaps Created
Total number of gaps:  164
Number of Linear Gaps filled: 9
Single gaps filled
Number of gaps with backup water level: 128
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot fill gap out of bounds
Cannot f